# FinePersonas による SFT データ合成

Susumu Ota  2025-02-02

## 合成データの重要性

TODO

## 合成データ作成の難しさ

TODO

## ペルソナとは

- 個人の特徴、背景、目標を詳細に記述したもので、多様なアイデンティティと経験を反映するようにデザインされている
- ペルソナの例
  - A network engineer with a focus on routing protocols and preparing for Cisco certification exams, particularly CCNA.
  - ルーティング・プロトコルに興味があり、シスコの認定試験(特に CCNA)の準備をしているネットワーク・エンジニア
- 生成するコンテンツに、特定の専門知識・キャリアパス・個人的な興味を導入し、より繊細でターゲットを絞ったコンテンツが生成可能
- https://huggingface.co/datasets/argilla/FinePersonas-v0.1 より引用

## Persona-Hub とは

<img src="https://github.com/tencent-ailab/persona-hub/blob/main/assets/persona_overview.png?raw=true" width="800px">

- テクニカルレポート: https://arxiv.org/abs/2406.20094
- 背景: 既存の合成データ⽣成⼿法(インスタンス駆動・キーポイント駆動)では合成データの多様さをスケールアップすることが困難
- 目的: ⼤規模なペルソナコレクションを使ってスケーラブルな合成データを⽣成する(ペルソナ駆動)
- 方法
  - Webデータからペルソナを抽出
    - Text-to-Persona
    - Persona-to-Persona
    - ペルソナ例
      - a moving company driver, a chemical kinetics researcher, a musician interested in audio processing
  - ペルソナを使って合成データを生成
    - Create **a math problem** with **a moving company driver**
    - Create **a math problem** with **a chemical kinetics researcher**
    - Create **a math problem** with **a musician interested in audio processing**

## FinePersonas とは

<img src="https://cdn-uploads.huggingface.co/production/uploads/6435d564a4bd75c62cc03701/5wTHwgijTUKFI5B-N7gEg.png" width="800px">

- 2024年9月にリリースされた合成テキスト生成のための2100万人の詳細なペルソナのオープンデータセット
- 合成データの豊富さ・多様性・特異性を高めることが可能
- FineWeb-Edu(教育関連Webページのデータセット)から抽出
- ライセンス: Llama 3.1 Community License Agreement
- 提供元: Argilla(高品質なデータセットを構築するためのコラボレーションツール)
- https://huggingface.co/datasets/argilla/FinePersonas-v0.1

## モジュールのインストール

`openai` モジュールと `vllm` モジュールのどちらか一方をインストールする必要があります。ローカルに GPU がある場合のみ `vllm` をインストールしてください。

In [1]:
# %pip install openai
# %pip install vllm
# %pip install datasets

## モジュールのインポート

In [2]:
from abc import ABC, abstractmethod
import json
from logging import DEBUG, INFO, StreamHandler, getLogger  # noqa: F401
import os  # noqa: F401
import pprint
import random
import re


from datasets import load_dataset
from openai import OpenAI

try:
    from vllm import LLM, SamplingParams  # type: ignore
except ImportError:
    print("No vllm module found. You can only use the OpenAI compatible APIs.")

/Users/ota/Documents/python/synthetic-data/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No vllm module found. You can only use the OpenAI compatible APIs.


## ログの設定

In [3]:
logging_level = DEBUG
# logging_level = INFO
logger = getLogger(__name__)
logger.setLevel(logging_level)
handler = StreamHandler()
handler.setLevel(logging_level)
logger.addHandler(handler)

pp = pprint.PrettyPrinter(width=119)

## 言語モデルの抽象化 (LanguageModel クラス)

OpenAI API と vLLM で統一してコードを書くために、`LanguageModel` という抽象クラスを作り、サブクラスで個別の処理を実装します。

- `__init__`: 言語モデルの初期化。推論用のパラメータ(`temperature`等)も設定。
- `__call__`: 推論の実行。入力は OpenAI messages 形式 (e.g. `[{"role": "user", "content": "Hello!"}]`) のリスト。出力は文字列のリスト。

In [4]:
class LanguageModel(ABC):
    def __init__(self, model_name: str, temperature=0.7, max_tokens=512, seed=0):
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.seed = seed
        logger.debug(f"model_name: {model_name}, temperature: {temperature}, max_tokens: {max_tokens}, seed: {seed}")

    @abstractmethod
    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        pass

## OpenAI API による LanguageModel の実装

OpenAI API を利用して言語モデルを実装します。`base_url` と `api_key` を設定することで、OpenAI 以外のサービスを利用することが可能です。

- OpenAIの場合: 環境変数 `OPENAI_API_KEY` に API キーを設定。
- NVIDIAの場合: `base_url="https://integrate.api.nvidia.com/v1", api_key=os.environ["NVIDIA_API_KEY"]` とした上で、環境変数 `NVIDIA_API_KEY` に API キーを設定。
- DeepInfraの場合: `base_url="https://api.deepinfra.com/v1/openai", api_key=os.environ["DEEPINFRA_API_KEY"]` とした上で、環境変数 `DEEPINFRA_API_KEY` に API キーを設定。

**Note: API キーは Notebook 内に直接書き込まないほうが安全です。出来ればシークレット管理サービス (e.g. GCP の Secret Manager) を利用するか、リスクを認識した上で環境変数を利用してください。**

In [5]:
class OpenAIAPI(LanguageModel):
    def __init__(self, model_name: str, temperature=0.7, max_tokens=512, seed=0, base_url=None, api_key=None):
        super().__init__(model_name, temperature=temperature, max_tokens=max_tokens, seed=seed)
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        logger.debug(f"base_url: {self.client.base_url}")

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        return [
            self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                seed=self.seed,
            ).choices[0].message.content or ""
            for messages in messages_batch
        ]

## vLLM を使った LanguageModel の実装

ローカルの GPU を使って推論する場合は vLLM を利用します。さらに推論に必要なパラメータがあれば `SamplingParams` の部分に追加してください。

In [6]:
class VLLMModel(LanguageModel):
    def __init__(self, model_name: str, temperature=0.7, max_tokens=512, seed=0, stop=[], repetition_penalty=1.0):
        super().__init__(model_name, temperature=temperature, max_tokens=max_tokens, seed=seed)
        self.vllm = LLM(model_name)
        self.sampling_params = SamplingParams(
            temperature=temperature, max_tokens=max_tokens, seed=seed, stop=stop, repetition_penalty=repetition_penalty
        )
        logger.debug(f"sampling_params: {self.sampling_params}")

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        tokenizer = self.vllm.get_tokenizer()
        prompts = [
            tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            for messages in messages_batch
        ]
        outputs = self.vllm.generate(prompts, sampling_params=self.sampling_params, use_tqdm=False)
        return [o.outputs[0].text for o in outputs]

## 言語モデルの初期化

以下のどれかのコメントを外して言語モデルを初期化します。`gpt-4o-mini` が最も安定している印象です。なお API 利用には料金が発生します。

一般に API の推論速度は遅いので、予備実験を API で行い、本実験は vLLM でバッチ処理するという使い方が現実的かもしれません。

API で大量のデータを生成する場合は、バッチ処理のオプションが用意されていればそれを利用することと、利用制限(rate limit等)の範囲内でリクエストの並列化を検討してください。

In [7]:
llm = OpenAIAPI("gpt-4o-mini", temperature=0.7, max_tokens=512, seed=0)  # You must set OPENAI_API_KEY in your environment variables.
# llm = OpenAIAPI("nvidia/llama-3.1-nemotron-70b-instruct", temperature=0.7, max_tokens=512, seed=0, base_url="https://integrate.api.nvidia.com/v1", api_key=os.environ["NVIDIA_API_KEY"])
# llm = OpenAIAPI("nvidia/nemotron-4-340b-instruct", temperature=0.7, max_tokens=512, seed=0, base_url="https://integrate.api.nvidia.com/v1", api_key=os.environ["NVIDIA_API_KEY"])
# llm = OpenAIAPI("nvidia/Llama-3.1-Nemotron-70B-Instruct", temperature=0.7, max_tokens=512, seed=0, base_url="https://api.deepinfra.com/v1/openai", api_key=os.environ["DEEPINFRA_API_KEY"])
# llm = VLLMModel("cyberagent/calm3-22b-chat", temperature=0.7, max_tokens=512, seed=0, stop=["<|endoftext|>", "<|im_end|>", "<|im_start|>"], repetition_penalty=1.1)

llm

model_name: gpt-4o-mini, temperature: 0.7, max_tokens: 512, seed: 0
base_url: https://api.openai.com/v1/


## 言語モデルの動作確認

ここまで設定できれば以下の推論が動作するはずです。もしエラーが出る場合は、言語モデルの初期化の設定を見直してください。

In [8]:
llm([[{"role": "user", "content": "Hello?"}]])

['Hello! How can I assist you today?']

## ペルソナデータの読み込み

ここからは実際のペルソナデータを読み込んで、対話データを生成します。`argilla/FinePersonas-v0.1` データセットを利用します。

FinePersonas データセットは巨大なデータセットですので、今回は一部分だけをダウンロードして利用します。読み込む際に `streaming=True` を必ずつけてください。

In [9]:
dataset = load_dataset("argilla/FinePersonas-v0.1", split="train", cache_dir="cache", streaming=True)
dataset

IterableDataset({
    features: ['id', 'persona', 'labels'],
    num_shards: 12
})

今回は先頭の100件を使って生成します。

In [10]:
personas = [data["persona"] for data in dataset.take(100)]
pp.pprint(personas)

['A professional R programmer or researcher, likely a data analyst or statistician, familiar with the intricacies of '
 'the R language and its debugging tools.',
 'A mental health professional, likely a licensed therapist or psychologist, with expertise in anxiety disorders and '
 'cognitive-behavioral therapy, whose work involves diagnosing and treating patients with various types of phobias, '
 'including specific phobia, social phobia, and agoraphobia.',
 'A space roboticist or aerospace engineer at a research institution focused on robotic space exploration.',
 'A Hebrew language scholar or instructor with a focus on biblical Hebrew and Jewish studies.',
 'A pediatrician or healthcare professional focused on educating parents about early childhood health, or a '
 'parent-to-be who is interested in learning about vaccinations for their unborn or newborn child.',
 'A Montessori elementary school teacher or an education administrator responsible for curriculum development and '
 'cla

In [11]:
def generate_instruction(llm: LanguageModel, persona: str) -> dict[str, str]:
    prompt = """\
Generate a prompt the persona below might ask to an AI assistant:

{persona}

Note:
1. Your response should always start with "プロンプト:".
2. 簡潔に日本語で回答してください。
"""
    return {
        "persona": persona,
        "text": llm([[{"role": "user", "content": prompt.format(persona=persona)}]])[0],
    }

In [12]:
response = generate_instruction(llm, random.choice(personas))
pp.pprint(response)

{'persona': 'A veterinarian focused on zoonotic diseases and preventative pet care, likely with a strong interest in '
            'public health and education.',
 'text': 'プロンプト: 最近の動物由来感染症についての最新の研究成果や予防策を教えてください。特にペットの健康管理に役立つ情報が知りたいです。'}


In [13]:
def generate_quora_post(llm: LanguageModel, persona: str) -> dict[str, str]:
    prompt = """\
Write a Quora post in the language, style, and personality of the following persona:

{persona}

Note:
1. Your response should always start with "タイトル:".
2. 簡潔に日本語で回答してください。
"""
    return {
        "persona": persona,
        "text": llm([[{"role": "user", "content": prompt.format(persona=persona)}]])[0],
    }

In [14]:
response = generate_quora_post(llm, random.choice(personas))
pp.pprint(response)

{'persona': 'A graduate student in international relations whose interests lie at the intersection of ethics, '
            'politics, and conflict studies, or a young academic researcher focused on the philosophy of war and its '
            'implications for global governance.',
 'text': 'タイトル: 戦争の倫理とグローバルガバナンスの交差点\n'
         '\n'
         '戦争と国際関係の研究は、単に歴史や事例を分析するだけではなく、倫理的な視点を考慮することが不可欠です。私たちが直面する現代の紛争は、単なる国家間の対立を超え、非国家主体やテロリズム、環境問題といった新たな要素を絡めています。\n'
         '\n'
         'このような文脈において、戦争の哲学は重要な役割を果たします。例えば、正義の戦争理論は、戦争の正当性を問う際に有用ですが、現実の戦争が常にこの理論に従って行われるわけではありません。倫理的なジレンマに直面する中で、私たちはどのようにグローバルガバナンスを再構築し、平和を持続可能にするのか、常に模索し続ける必要があります。\n'
         '\n'
         '私たちの研究が、未来の国際社会においてより良い選択を導く手助けとなることを願っています。戦争の本質を理解し、倫理的な視点を持ちながら、国際関係の複雑さを解き明かしていくことが、私たちの使命です。'}


## Persona-Hub による SFT データ合成

ここではマルチターンの対話データを生成します。具体的には、

- 質問1 (`Q1`): ユーザが質問をする (この部分を Persona-Hub 手法で生成)
- 解答1 (`A1`): アシスタントが解答する
- 質問2 (`Q2`): ユーザが追加質問をする
- 解答2 (`A2`): アシスタントが追加質問に解答する

という4つの対話データを生成します。このうち `Q1` の生成に Persona-Hub 手法を適用します。それ以降の対話データは、若干の工夫をしますが基本的に言語モデルの生成に任せます。

## Q1 の生成

まず、Persona-Hub 手法を使って `Q1` の生成を行います。

[元論文のコード](https://github.com/tencent-ailab/persona-hub/blob/main/code/prompt_templates.py)では、数学の問題を生成するプロンプトとして以下が紹介されています。

```python
math_template = '''Create a math problem related to the following persona:

{persona}

Note:

1. The math problem should be challenging and involve advanced mathematical skills and knowledge. Only top talents can solve it correctly.
2. You should make full use of the persona description to create the math problem to ensure that the math problem is unique and specific to the persona.
3. Your response should always start with "Math problem:". Your response should not include a solution to the created math problem.
4. Your created math problem should include no more than 2 sub-problems.
'''
```

これを参考に、プロンプトの一部を修正/パラメータ化して使います。修正点は以下の通りです。

- 一部の単語をパラメータ化
  - タスクの種類: `task` (e.g. "math", "reasoning", "coding")
  - トピックの種類: `topic` (e.g. "persona", "topic")
  - トピックの内容: `item` (e.g. "SF作家", "経営コンサルタント")
  - 対象者: `target` (e.g. "grade school student", "graduate student")
- 難易度を調整
  - 元のプロンプトはかなり難易度の高い問題を生成するような表現(`challenging`, `advanced`, `top talents`)となっていたため、難易度を下げるように表現を修正(`simple`, `basic`, `average`)
- 小問を生成しないよう修正
  - 小問を生成すると対話が複雑になりすぎるため、小問を生成しないように修正(`sub-problems`の行を削除)。代わりにマルチターンで追加質問をするようにする。
- 日本語を出力
  - 元のプロンプトは英語を出力するようになっているため、日本語を出力するように促すプロンプトを追加

上記の修正をしたプロンプトを OpenAI messages 形式で出力する関数を作成します。

利用する言語モデルで推奨されているシステムプロンプトがある場合は、それを指定します(コメントアウトされた `{"role": "system", "content": "..."}` の部分)。

In [15]:
def get_q1_prompt(task: str, topic: str, item: str, target: str) -> list[dict[str, str]]:
    Q1_PROMPT_TEMPLATE = """Create a {task} problem related to the following {topic}:

{item}

Note:

1. The {task} problem should be simple and involve basic {task} skills and knowledge. Any average {target} can solve it correctly.
2. You should make full use of the {topic} description to create the {task} problem to ensure that the {task} problem is unique and specific to the {topic}.
3. Your response should always start with "問題:". Your response should not include a solution to the created {task} problem.
4. 簡潔に日本語で回答してください。
"""
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": Q1_PROMPT_TEMPLATE.format(task=task, topic=topic, item=item, target=target)},
    ]

実行して動作を確認します。

In [16]:
q1_prompt = get_q1_prompt("math", "persona", random.choice(personas), "grade school student")
pp.pprint(q1_prompt)

[{'content': 'Create a math problem related to the following persona:\n'
             '\n'
             'An elementary school teacher or educator focused on creating and compiling educational content for '
             'young students, likely in a science, reading, or early childhood development curriculum.\n'
             '\n'
             'Note:\n'
             '\n'
             '1. The math problem should be simple and involve basic math skills and knowledge. Any average grade '
             'school student can solve it correctly.\n'
             '2. You should make full use of the persona description to create the math problem to ensure that the '
             'math problem is unique and specific to the persona.\n'
             '3. Your response should always start with "問題:". Your response should not include a solution to the '
             'created math problem.\n'
             '4. 簡潔に日本語で回答してください。\n',
  'role': 'user'}]


問題なさそうなので、言語モデルで推論して Q1 を生成します。

In [17]:
q1s = llm([get_q1_prompt("math", "persona", random.choice(personas), "grade school student")])
pp.pprint(q1s)

['問題: 天文学に熱中しているアマチュア天文学者の田中さんは、毎晩星を観察しています。彼は、1週間の間に合計で35個の星を観察しました。もし彼が毎晩同じ数の星を観察したとしたら、彼は1晩に何個の星を観察したのでしょうか？']


引数を変更して難易度の高い問題を生成します。

In [18]:
q1s = llm([get_q1_prompt("advanced math", "persona", random.choice(personas), "graduate student")])
pp.pprint(q1s)

['問題:  \n'
 'あるヘブライ語の研究者が、聖書のテキスト中に出現する動詞の数を調査しています。彼は、特定の文脈での動詞の出現率が全体の文の中で 30% であることを発見しました。彼が調査した 1500 '
 '文の中で、動詞が出現した文の数を求めなさい。さらに、彼が調査した文の中で動詞が出現しなかった文の割合が 70% である場合、動詞が出現しなかった文の数も求めなさい。']


問題さなさそうです。

次に、`問題:` など余分な部分をルールベース(正規表現)で削除します。言語モデルによっては、解答やヒントを書いてしまうものがありますので、もしそれらが出力されるのであればここで削除しておきます。

**Note: この関数が複雑になり過ぎる場合は、プロンプトの見直しや言語モデルの変更を検討した方が良いかもしれません。**

In [19]:
def filter_q1(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^問題[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^Problem[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"\n答え[：:].*", "", content, flags=re.DOTALL)
    content = re.sub(r"\n[解回]答[：:].*", "", content, flags=re.DOTALL)
    content = re.sub(r"\n[Aa]nswer[：:].*", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

実行して動作を確認します。

In [20]:
filter_q1(q1s[0])

'あるヘブライ語の研究者が、聖書のテキスト中に出現する動詞の数を調査しています。彼は、特定の文脈での動詞の出現率が全体の文の中で 30% であることを発見しました。彼が調査した 1500 文の中で、動詞が出現した文の数を求めなさい。さらに、彼が調査した文の中で動詞が出現しなかった文の割合が 70% である場合、動詞が出現しなかった文の数も求めなさい。'

In [21]:
filter_q1("問題: ここは問題です。\n\n解答: ここは答えです。")

'ここは問題です。'

問題なさそうです。

作成した関数を組み合わせて、バッチ処理で `Q1` を生成する関数を作成します。

In [22]:
def generate_q1(
    llm: LanguageModel, tasks: list[str], topics: list[str], items: list[str], targets: list[str]
) -> list[str]:
    return [
        filter_q1(q1)
        for q1 in llm(
            [
                get_q1_prompt(task, topic, item, target)
                for task, topic, item, target in zip(tasks, topics, items, targets)
            ]
        )
    ]

実行して動作を確認します。

In [23]:
q1s = generate_q1(
    llm,
    ["math", "advanced math"],
    ["persona", "persona"],
    [random.choice(personas), random.choice(personas)],
    ["grade school student", "graduate student"],
)
pp.pprint(q1s)

['ある宇宙ロボット工学者が、火星探査のために新しいロボットを設計しています。このロボットは、1時間で10キロメートルの速度で移動します。もしロボットが火星の表面を30時間移動し続けた場合、ロボットは合計で何キロメートル移動することになりますか？',
 'ある気象学者が、過去の大洪水のデータを基にして、特定の地域での降水量の変動を解析しています。この地域では、過去100年間の年平均降水量が次のように与えられています：\n'
 '\n'
 '\\[ P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}t\\right) \\]\n'
 '\n'
 'ここで、\\( P(t) \\)は年\\( t \\)における降水量（mm）、\\( t \\)は年数（0から99まで）を表します。\n'
 '\n'
 'この降水量の変動を解析するために、以下の問いに答えてください：\n'
 '\n'
 '1. 10年ごとの平均降水量を求めなさい。\n'
 '2. 降水量が最大となる年を特定し、その年の降水量を計算しなさい。\n'
 '3. 過去100年間の降水量の標準偏差を計算しなさい。']


問題なさそうです。ここまでで `Q1` を生成する関数の実装が完了しました。

## A1 の生成

次に、`A1` の生成を行います。ここでは基本的に `Q1` を入力して、長さ指定(`簡潔に`)と、出力言語の指定(`日本語で`)をして言語モデルに生成させます。言語モデルによっては、表現を微調整する必要があるかもしれません。

実行の流れは `Q1` と同様ですので、ここではコードのみ記載します。

In [24]:
def get_a1_prompt(q1: str) -> list[dict]:
    A1_PROMPT_TEMPLATE = "{q1}\n\n簡潔に日本語で回答してください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": A1_PROMPT_TEMPLATE.format(q1=q1)},
    ]

In [25]:
a1_prompt = get_a1_prompt(q1s[0])
pp.pprint(a1_prompt)

[{'content': 'ある宇宙ロボット工学者が、火星探査のために新しいロボットを設計しています。このロボットは、1時間で10キロメートルの速度で移動します。もしロボットが火星の表面を30時間移動し続けた場合、ロボットは合計で何キロメートル移動することになりますか？\n'
             '\n'
             '簡潔に日本語で回答してください。',
  'role': 'user'}]


In [26]:
a1s = llm([get_a1_prompt(q1s[0])])
pp.pprint(a1s)

['ロボットは1時間で10キロメートル移動するので、30時間で移動する距離は次のように計算できます。\n\n10キロメートル/時間 × 30時間 = 300キロメートル\n\nしたがって、ロボットは合計で300キロメートル移動します。']


In [27]:
a1s = llm([get_a1_prompt(q1s[1])])
pp.pprint(a1s)

['1. **10年ごとの平均降水量を求める**  \n'
 '降水量の式は \\( P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}t\\right) \\) '
 'です。10年ごとの平均降水量を求めるために、各10年間の降水量を計算し、それを平均します。\n'
 '\n'
 '- **0年から9年**:  \n'
 '  \\(\\text{平均} = \\frac{1}{10} \\sum_{t=0}^{9} P(t) = \\frac{1}{10} \\sum_{t=0}^{9} \\left( 800 + 50 '
 '\\sin\\left(\\frac{2\\pi}{10}t\\right) \\right)\\)  \n'
 '  \\(\\text{平均} = 800 + 5 \\sum_{t=0}^{9} \\sin\\left(\\frac{2\\pi}{10}t\\right) \\)  \n'
 '  \\(\\sum_{t=0}^{9} \\sin\\left(\\frac{2\\pi}{10}t\\right) = 0\\) なので、  \n'
 '  \\(\\text{平均} = 800 \\, \\text{mm}\\)\n'
 '\n'
 '同様に、10年ごとに計算すると、\n'
 '\n'
 '- **10年から19年**: 800 mm\n'
 '- **20年から29年**: 800 mm\n'
 '- **30年から39年**: 800 mm\n'
 '- **40年から49年**: 800 mm\n'
 '- **50年から59年**: 800 mm\n'
 '- **60年から69年**: 800 mm\n'
 '- **70年から79年**: 800 mm\n'
 '- **80年から89年**: 800 mm\n'
 '- **90年から99年**: 800 mm\n'
 '\n'
 'したがって、各10年ごとの平均降水量はすべて800 mmです。\n'
 '\n'
 '2. **降水量が最大となる年を特定し、その年の降水量を計算**  \n'
 '降水量が最大になるのは、\\(\\sin\\)の値が1になるときです。  \n'
 '\\(\\sin\\left(\\f

In [28]:
def filter_a1(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^答え[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[解回]答[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[Aa]nswer[：:]", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

In [29]:
filter_a1("解答: ここは答えです。")

'ここは答えです。'

In [30]:
def generate_a1(llm: LanguageModel, q1s: list[str]) -> list[str]:
    return [filter_a1(a1) for a1 in llm([get_a1_prompt(q1) for q1 in q1s])]

In [31]:
a1s = generate_a1(llm, q1s)
pp.pprint(a1s)

['ロボットは1時間で10キロメートル移動するので、30時間で移動する距離は次のように計算できます。\n\n10キロメートル/時間 × 30時間 = 300キロメートル\n\nしたがって、ロボットは合計で300キロメートル移動します。',
 '1. **10年ごとの平均降水量を求める**  \n'
 '降水量の式は \\( P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}t\\right) \\) です。10年ごとの平均降水量を求めるために、各10年間の降水量を計算します。\n'
 '\n'
 '- \\( t = 0 \\) から \\( 9 \\):\n'
 '  \\[\n'
 '  P(0) = 800 + 50 \\sin(0) = 800 \\\\\n'
 '  P(1) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}\\right) \\approx 800 + 50 \\times 0.309 = 815.5 \\\\\n'
 '  P(2) = 800 + 50 \\sin\\left(\\frac{4\\pi}{10}\\right) \\approx 800 + 50 \\times 0.588 = 814 \\\\\n'
 '  P(3) = 800 + 50 \\sin\\left(\\frac{6\\pi}{10}\\right) \\approx 800 + 50 \\times 0.809 = 820.5 \\\\\n'
 '  P(4) = 800 + 50 \\sin\\left(\\frac{8\\pi}{10}\\right) \\approx 800 + 50 \\times 0.951 = 827.5 \\\\\n'
 '  P(5) = 800 + 50 \\sin(2\\pi) = 800 \\\\\n'
 '  P(6) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10} \\times 6\\right) \\approx 800 - 50 \\times 0.951 = 772.5 \\\\\n'
 '  P(7) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10} \\time

## Q2 の生成

次に、追加質問 `Q2` の生成を行います。

- まず `Q1` に関連する質問であることを強調します(`前述の問題をより理解するために`の部分)。これが無いと `Q2` で新たな別の質問を生成してしまうことがあります。
- 次に `問題の一部を変更したり、条件を追加しても良いです` という部分で、`Q1` の問題の一部を変更することを促します。これにより、`Q2` が `Q1` に関連する質問となる確率を高めることができます。
- 最後に答えを含まないように注意を促します(`決して答えを含めないでください`の部分)。

In [32]:
def get_q2_prompt(q1: str, a1: str) -> list[dict[str, str]]:
    Q2_PROMPT_TEMPLATE = "前述の問題をより理解するために、簡潔な追加の質問を一つ作ってください。問題の一部を変更したり、条件を追加しても良いです。追加の質問だけを書き、決して答えを含めないでください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": q1},
        {"role": "assistant", "content": a1},
        {"role": "user", "content": Q2_PROMPT_TEMPLATE},
    ]

In [33]:
q2_prompt = get_q2_prompt(q1s[0], a1s[0])
pp.pprint(q2_prompt)

[{'content': 'ある宇宙ロボット工学者が、火星探査のために新しいロボットを設計しています。このロボットは、1時間で10キロメートルの速度で移動します。もしロボットが火星の表面を30時間移動し続けた場合、ロボットは合計で何キロメートル移動することになりますか？',
  'role': 'user'},
 {'content': 'ロボットは1時間で10キロメートル移動するので、30時間で移動する距離は次のように計算できます。\n'
             '\n'
             '10キロメートル/時間 × 30時間 = 300キロメートル\n'
             '\n'
             'したがって、ロボットは合計で300キロメートル移動します。',
  'role': 'assistant'},
 {'content': '前述の問題をより理解するために、簡潔な追加の質問を一つ作ってください。問題の一部を変更したり、条件を追加しても良いです。追加の質問だけを書き、決して答えを含めないでください。',
  'role': 'user'}]


In [34]:
q2s = llm([get_q2_prompt(q1s[0], a1s[0])])
pp.pprint(q2s)

['ロボットが火星の表面を30時間移動した後、休憩のために5時間停止した場合、ロボットが移動を再開した後にさらに10時間移動した場合、合計で何キロメートル移動することになりますか？']


In [35]:
def filter_q2(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^追加の質問[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^質問[：:]", "", content, flags=re.DOTALL)
    content = content.strip()
    return content

In [36]:
def generate_q2(llm: LanguageModel, q1s: list[str], a1s: list[str]) -> list[str]:
    return [filter_q2(q2) for q2 in llm([get_q2_prompt(q1, a1) for q1, a1 in zip(q1s, a1s)])]

In [37]:
q2s = generate_q2(llm, q1s, a1s)
pp.pprint(q2s)

['ロボットが1時間に12キロメートルの速度で移動する場合、30時間で合計何キロメートル移動することになりますか？',
 '降水量の式が以下のように変更されました：\n'
 '\n'
 '\\[ P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{15}t + \\frac{\\pi}{4}\\right) \\]\n'
 '\n'
 'この新しい降水量の式に基づき、次の問いに答えなさい：\n'
 '\n'
 '4. 10年ごとの降水量の変動幅（最大降水量と最小降水量の差）を求めなさい。']


一旦、ここまでの対話データの流れを確認します。

In [38]:
pp.pprint(list(zip(q1s, a1s, q2s))[0])

('ある宇宙ロボット工学者が、火星探査のために新しいロボットを設計しています。このロボットは、1時間で10キロメートルの速度で移動します。もしロボットが火星の表面を30時間移動し続けた場合、ロボットは合計で何キロメートル移動することになりますか？',
 'ロボットは1時間で10キロメートル移動するので、30時間で移動する距離は次のように計算できます。\n\n10キロメートル/時間 × 30時間 = 300キロメートル\n\nしたがって、ロボットは合計で300キロメートル移動します。',
 'ロボットが1時間に12キロメートルの速度で移動する場合、30時間で合計何キロメートル移動することになりますか？')


In [39]:
pp.pprint(list(zip(q1s, a1s, q2s))[1])

('ある気象学者が、過去の大洪水のデータを基にして、特定の地域での降水量の変動を解析しています。この地域では、過去100年間の年平均降水量が次のように与えられています：\n'
 '\n'
 '\\[ P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}t\\right) \\]\n'
 '\n'
 'ここで、\\( P(t) \\)は年\\( t \\)における降水量（mm）、\\( t \\)は年数（0から99まで）を表します。\n'
 '\n'
 'この降水量の変動を解析するために、以下の問いに答えてください：\n'
 '\n'
 '1. 10年ごとの平均降水量を求めなさい。\n'
 '2. 降水量が最大となる年を特定し、その年の降水量を計算しなさい。\n'
 '3. 過去100年間の降水量の標準偏差を計算しなさい。',
 '1. **10年ごとの平均降水量を求める**  \n'
 '降水量の式は \\( P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}t\\right) \\) です。10年ごとの平均降水量を求めるために、各10年間の降水量を計算します。\n'
 '\n'
 '- \\( t = 0 \\) から \\( 9 \\):\n'
 '  \\[\n'
 '  P(0) = 800 + 50 \\sin(0) = 800 \\\\\n'
 '  P(1) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}\\right) \\approx 800 + 50 \\times 0.309 = 815.5 \\\\\n'
 '  P(2) = 800 + 50 \\sin\\left(\\frac{4\\pi}{10}\\right) \\approx 800 + 50 \\times 0.588 = 814 \\\\\n'
 '  P(3) = 800 + 50 \\sin\\left(\\frac{6\\pi}{10}\\right) \\approx 800 + 50 \\times 0.809 = 820.5 \\\\\n'
 '  P(4) = 800 + 50 \\sin\\left(\\frac{8\\pi}{10}\\right) 

問題なさそうです。

## A2 の生成

最後に、`A2` の生成を行います。基本的に `A1` の生成と同様ですので、ここではコードのみ記載します。

In [40]:
def get_a2_prompt(q1: str, a1: str, q2: str) -> list[dict[str, str]]:
    A2_PROMPT_TEMPLATE = "{q2}\n\n簡潔に日本語で回答してください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": q1},
        {"role": "assistant", "content": a1},
        {"role": "user", "content": A2_PROMPT_TEMPLATE.format(q2=q2)},
    ]

In [41]:
a2_prompt = get_a2_prompt(q1s[0], a1s[0], q2s[0])
pp.pprint(a2_prompt)

[{'content': 'ある宇宙ロボット工学者が、火星探査のために新しいロボットを設計しています。このロボットは、1時間で10キロメートルの速度で移動します。もしロボットが火星の表面を30時間移動し続けた場合、ロボットは合計で何キロメートル移動することになりますか？',
  'role': 'user'},
 {'content': 'ロボットは1時間で10キロメートル移動するので、30時間で移動する距離は次のように計算できます。\n'
             '\n'
             '10キロメートル/時間 × 30時間 = 300キロメートル\n'
             '\n'
             'したがって、ロボットは合計で300キロメートル移動します。',
  'role': 'assistant'},
 {'content': 'ロボットが1時間に12キロメートルの速度で移動する場合、30時間で合計何キロメートル移動することになりますか？\n\n簡潔に日本語で回答してください。', 'role': 'user'}]


In [42]:
a2s = llm([get_a2_prompt(q1, a1, q2) for q1, a1, q2 in zip(q1s, a1s, q2s)])
pp.pprint(a2s)

['ロボットは30時間で合計360キロメートル移動します。',
 '新しい降水量の式は \\( P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{15}t + \\frac{\\pi}{4}\\right) \\) '
 'です。この式に基づいて、降水量の変動幅を求めます。\n'
 '\n'
 '1. **最大値を求める**:\n'
 '   \\[\n'
 '   \\sin\\left(\\frac{2\\pi}{15}t + \\frac{\\pi}{4}\\right) \\text{ の最大値は } 1 です。\n'
 '   \\]\n'
 '   したがって、最大降水量 \\( P_{\\text{max}} \\) は：\n'
 '   \\[\n'
 '   P_{\\text{max}} = 800 + 50 \\cdot 1 = 850 \\text{ mm}\n'
 '   \\]\n'
 '\n'
 '2. **最小値を求める**:\n'
 '   \\[\n'
 '   \\sin\\left(\\frac{2\\pi}{15}t + \\frac{\\pi}{4}\\right) \\text{ の最小値は } -1 です。\n'
 '   \\]\n'
 '   したがって、最小降水量 \\( P_{\\text{min}} \\) は：\n'
 '   \\[\n'
 '   P_{\\text{min}} = 800 + 50 \\cdot (-1) = 750 \\text{ mm}\n'
 '   \\]\n'
 '\n'
 '3. **変動幅を求める**:\n'
 '   \\[\n'
 '   \\text{変動幅} = P_{\\text{max}} - P_{\\text{min}} = 850 - 750 = 100 \\text{ mm}\n'
 '   \\]\n'
 '\n'
 'したがって、10年ごとの降水量の変動幅は **100 mm** です。']


In [43]:
def filter_a2(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^答え[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[解回]答[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[Aa]nswer[：:]", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

In [44]:
def generate_a2(llm: LanguageModel, q1s: list[str], a1s: list[str], q2s: list[str]) -> list[str]:
    return [filter_a2(a2) for a2 in llm([get_a2_prompt(q1, a1, q2) for q1, a1, q2 in zip(q1s, a1s, q2s)])]

In [45]:
a2s = generate_a2(llm, q1s, a1s, q2s)
pp.pprint(a2s)

['ロボットは30時間で合計360キロメートル移動します。',
 '新しい降水量の式は \\( P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{15}t + \\frac{\\pi}{4}\\right) \\) です。\n'
 '\n'
 'まず、降水量の最大値と最小値を求めます。\n'
 '\n'
 '- 最大値は、\\(\\sin\\) の最大値が 1 のときに達成されます：\n'
 '  \\[\n'
 '  P_{\\text{max}} = 800 + 50 \\times 1 = 850 \\, \\text{mm}\n'
 '  \\]\n'
 '\n'
 '- 最小値は、\\(\\sin\\) の最小値が -1 のときに達成されます：\n'
 '  \\[\n'
 '  P_{\\text{min}} = 800 + 50 \\times (-1) = 750 \\, \\text{mm}\n'
 '  \\]\n'
 '\n'
 '次に、変動幅を求めます：\n'
 '\\[\n'
 '\\text{変動幅} = P_{\\text{max}} - P_{\\text{min}} = 850 - 750 = 100 \\, \\text{mm}\n'
 '\\]\n'
 '\n'
 'したがって、10年ごとの降水量の変動幅は **100 mm** です。']


再度ここまでの対話データの流れを確認します。

In [46]:
pp.pprint(list(zip(q1s, a1s, q2s, a2s))[0])

('ある宇宙ロボット工学者が、火星探査のために新しいロボットを設計しています。このロボットは、1時間で10キロメートルの速度で移動します。もしロボットが火星の表面を30時間移動し続けた場合、ロボットは合計で何キロメートル移動することになりますか？',
 'ロボットは1時間で10キロメートル移動するので、30時間で移動する距離は次のように計算できます。\n\n10キロメートル/時間 × 30時間 = 300キロメートル\n\nしたがって、ロボットは合計で300キロメートル移動します。',
 'ロボットが1時間に12キロメートルの速度で移動する場合、30時間で合計何キロメートル移動することになりますか？',
 'ロボットは30時間で合計360キロメートル移動します。')


In [47]:
pp.pprint(list(zip(q1s, a1s, q2s, a2s))[1])

('ある気象学者が、過去の大洪水のデータを基にして、特定の地域での降水量の変動を解析しています。この地域では、過去100年間の年平均降水量が次のように与えられています：\n'
 '\n'
 '\\[ P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}t\\right) \\]\n'
 '\n'
 'ここで、\\( P(t) \\)は年\\( t \\)における降水量（mm）、\\( t \\)は年数（0から99まで）を表します。\n'
 '\n'
 'この降水量の変動を解析するために、以下の問いに答えてください：\n'
 '\n'
 '1. 10年ごとの平均降水量を求めなさい。\n'
 '2. 降水量が最大となる年を特定し、その年の降水量を計算しなさい。\n'
 '3. 過去100年間の降水量の標準偏差を計算しなさい。',
 '1. **10年ごとの平均降水量を求める**  \n'
 '降水量の式は \\( P(t) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}t\\right) \\) です。10年ごとの平均降水量を求めるために、各10年間の降水量を計算します。\n'
 '\n'
 '- \\( t = 0 \\) から \\( 9 \\):\n'
 '  \\[\n'
 '  P(0) = 800 + 50 \\sin(0) = 800 \\\\\n'
 '  P(1) = 800 + 50 \\sin\\left(\\frac{2\\pi}{10}\\right) \\approx 800 + 50 \\times 0.309 = 815.5 \\\\\n'
 '  P(2) = 800 + 50 \\sin\\left(\\frac{4\\pi}{10}\\right) \\approx 800 + 50 \\times 0.588 = 814 \\\\\n'
 '  P(3) = 800 + 50 \\sin\\left(\\frac{6\\pi}{10}\\right) \\approx 800 + 50 \\times 0.809 = 820.5 \\\\\n'
 '  P(4) = 800 + 50 \\sin\\left(\\frac{8\\pi}{10}\\right) 

面白い問題かどうかはともかく、問題なさそうです。これで `A2` の生成も完了しました。

## マルチターンの対話データの生成

これまで作成した関数を組み合わせて、マルチターンの対話データを生成する関数を作成します。結果は OpenAI messages 形式で出力します。

In [48]:
def synthesis_multi_turn_qa(
    llm: LanguageModel, tasks: list[str], topics: list[str], items: list[str], targets: list[str]
) -> list[dict[str, str | list[dict[str, str]]]]:
    q1s = generate_q1(llm, tasks, topics, items, targets)
    a1s = generate_a1(llm, q1s)
    q2s = generate_q2(llm, q1s, a1s)
    a2s = generate_a2(llm, q1s, a1s, q2s)
    return [
        {
            "messages": [
                {"role": "user", "content": q1},
                {"role": "assistant", "content": a1},
                {"role": "user", "content": q2},
                {"role": "assistant", "content": a2},
            ],
            "task": task,
            "topic": topic,
            "item": item,
            "target": target,
        }
        for task, topic, item, target, q1, a1, q2, a2 in zip(tasks, topics, items, targets, q1s, a1s, q2s, a2s)
    ]

実行して動作を確認します。

In [49]:
qas = synthesis_multi_turn_qa(
    llm,
    ["math"],
    ["persona"],
    [random.choice(personas)],
    ["grade school student"],
)
pp.pprint(qas)

[{'item': 'An amateur astronomer and science educator with a focus on celestial events and a flair for sharing '
          'complex information with a general audience.',
  'messages': [{'content': 'アマチュア天文学者であるあなたは、星座の観察会を開催しています。参加者は合計で24人で、5人のグループに分けることにしました。すべてのグループが同じ人数になるようにした場合、何グループできますか？また、各グループには何人の参加者がいますか？',
                'role': 'user'},
               {'content': '24人を5人のグループに分けると、4グループできます。各グループには6人の参加者がいます。', 'role': 'assistant'},
               {'content': '参加者が30人に増えた場合、何グループできますか？各グループには何人の参加者がいますか？', 'role': 'user'},
               {'content': '参加者が30人の場合、6グループできます。各グループには5人の参加者がいます。', 'role': 'assistant'}],
  'target': 'grade school student',
  'task': 'math',
  'topic': 'persona'}]


この関数をバッチ処理で実行し、結果を JSONL ファイルに保存する関数を作成します。`task`, `topic`, `item`, `target` は、与えられた引数からランダムに選択して、対話データを生成します。

In [50]:
def run_synthesis_multi_turn_qa(
    llm: LanguageModel,
    task_list: list[str],
    topic_list: list[str],
    item_list: list[str],
    target_list: list[str],
    num_samples: int = 1,
    batch_size: int = 1,
    output_jsonl: str = "output.jsonl",
):
    with open(output_jsonl, "a", encoding="utf-8") as f:
        for i in range(num_samples // batch_size):
            tasks = [random.choice(task_list) for _ in range(batch_size)]
            topics = [random.choice(topic_list) for _ in range(batch_size)]
            items = [random.choice(item_list) for _ in range(batch_size)]
            targets = [random.choice(target_list) for _ in range(batch_size)]
            assert len(tasks) == len(topics) == len(items) == len(targets) == batch_size
            qas = synthesis_multi_turn_qa(llm, tasks, topics, items, targets)
            for qa in qas:
                f.write(json.dumps(qa, ensure_ascii=False) + "\n")
                logger.debug(pp.pformat(qa))

ペルソナデータから対話データを生成します。

In [51]:
run_synthesis_multi_turn_qa(
    llm,
    ["math", "arithmetic", "basic math", "basic arithmetic"],
    ["persona"],
    personas,
    ["grade school student"],
    1,
    1,
    "output.jsonl",
)

{'item': 'A psychiatrist, possibly sub-specializing in mood disorders.',
 'messages': [{'content': 'ある精神科医が、気分障害に関する研究を行っています。彼は、患者10人の気分の変化を観察しました。そのうち、5人の患者が気分が改善し、3人の患者が気分が悪化しました。残りの患者は変化がなかったと報告しています。 \n'
                          '\n'
                          'この精神科医の観察した患者の中で、気分が変化しなかった患者の人数は何人ですか？',
               'role': 'user'},
              {'content': '気分が変化しなかった患者の人数は2人です。', 'role': 'assistant'},
              {'content': '気分が悪化した患者の中で、どのくらいの割合が治療を受けていたかを知ることは、研究の結果にどのような影響を与えるでしょうか？', 'role': 'user'},
              {'content': '気分が悪化した患者の治療を受けていた割合を知ることは、治療の効果や治療法の有効性を評価するのに重要です。この情報があれば、治療が気分悪化にどのように寄与したかを分析でき、さらに改善策や新たなアプローチを考える材料になります。逆に、治療を受けていない患者が多い場合は、治療の必要性や重要性を示す根拠となります。',
               'role': 'assistant'}],
 'target': 'grade school student',
 'task': 'math',
 'topic': 'persona'}


難易度を上げて生成します。

In [53]:
run_synthesis_multi_turn_qa(
    llm,
    ["advanced math"],
    ["persona"],
    personas,
    ["graduate student"],
    1,
    1,
    "output.jsonl",
)

{'item': 'A physics historian focused on documenting and promoting the development of major scientific discoveries '
         'and the lives of influential physicists.',
 'messages': [{'content': '物理学の歴史を研究するあなたは、特定の物理学者の業績を文書化するために、彼が発表した論文の数とそれに関連する引用数を分析しています。ある物理学者は、彼のキャリアの初期に10本の論文を発表し、各論文は平均して15回引用されました。その後の10年間で、彼は毎年2本の論文を発表し続け、各論文が平均して20回引用されるようになりました。\n'
                          '\n'
                          '1. 彼のキャリアの初期における論文の総引用数を求めてください。\n'
                          '2. 10年間で発表された論文の総数とその総引用数を求めてください。\n'
                          '3. 彼のキャリア全体を通じた論文の総数と総引用数を求めてください。',
               'role': 'user'},
              {'content': '1. 彼のキャリアの初期における論文の総引用数は、10本の論文 × 15回引用 = 150回です。\n'
                          '\n'
                          '2. 10年間で発表された論文の総数は、毎年2本 × 10年 = 20本です。また、これらの論文の総引用数は、20本 × 20回引用 = 400回です。\n'
                          '\n'
                          '3. 彼のキャリア全体を通じた論文の総数は、初期の10本 + 10年間での20本 = 30本です。総引用数は、初期の150回 + 10年間での400回 = 550回です。',
               

以上